In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import six
import gzip
import math
import os
import sys
import json
from collections import defaultdict

from rdkit import Chem, RDLogger
from rdkit.Chem import AllChem, rdMolDescriptors, RDConfig
from rdkit.Chem.SpacialScore import SPS

sys.path.append(os.path.join(RDConfig.RDContribDir, 'SA_Score'))
import sascorer
sys.path.append(os.path.join(RDConfig.RDContribDir, 'ChiralPairs'))
import ChiralDescriptors

### Functions of other 8 metrics in the paper apart from MC1 & MC2

#### 1.FCFP4:

In [2]:
# TODO:The sum of on-bits in an ECFP4 as a complexity measure

def calculate_fcfp4_complexity(smiles):
    # Convert SMILES to RDKit molecule object
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        print(f"Invalid SMILES: {smiles}")
        return None
    
    
    # Suppress RDKit warnings
    RDLogger.DisableLog('rdApp.*')
    
    # Generate FCFP_4 fingerprint (functional-based ECFP4)
    # Use the 'useFeatures=True' argument to focus on functional groups
    fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=2048, useFeatures=True)

    # Calculate the complexity score by counting on-bits
    complexity_score = fp.GetNumOnBits()

    return complexity_score

In [3]:
calculate_fcfp4_complexity("COC1=C(O)C=C(CC(=O)O)C=C1Br")

29

#### 2.Data Warrior:

In [4]:
# You can use the [**DataWarrior**](https://openmolecules.org/datawarrior/download.html) software to calculate the complexity

#### 3.Böttcher:

In [5]:
# Current failures: Does not distinguish between cyclopentyl and pentyl (etc.)
#                   and so unfairly underestimates complexity.

def GetChemicalNonequivs(atom, themol):
    num_unique_substituents = 0
    substituents = [[],[],[],[]]
    for item,key in enumerate(ChiralDescriptors.determineAtomSubstituents(atom.GetIdx(), themol, Chem.GetDistanceMatrix(themol))[0]):
        for subatom in ChiralDescriptors.determineAtomSubstituents(atom.GetIdx(), themol, Chem.GetDistanceMatrix(themol))[0][key]:
            substituents[item].append(themol.GetAtomWithIdx(subatom).GetSymbol())
            num_unique_substituents = len(set(tuple(tuple(substituent) for substituent in substituents if substituent)))
            #
            # Logic to determine e.g. whether repeats of CCCCC are cyclopentyl and pentyl or two of either
            #
    return num_unique_substituents

#
# The number of different non-hydrogen elements or isotopes (including deuterium
# and tritium) in the atom's microenvironment.
#
# CH4 - the carbon has e_i of 1
# Carbonyl carbon of an amide e.g. CC(=O)N e_i = 3
#     while N and O have e_i = 2
#
def GetBottcherLocalDiversity(atom):
    neighbors = []
    for neighbor in atom.GetNeighbors():
        neighbors.append(str(neighbor.GetSymbol()))
    if atom.GetSymbol() in set(neighbors):
        return len(set(neighbors))
    else:
        return len(set(neighbors))+1

#
# RDKit marks atoms where there is potential for isomerization with a tag 
# called _CIPCode. If it exists for an atom, note that S = 2, otherwise 1. 
def GetNumIsomericPossibilities(atom):
    try:
        if(atom.GetProp('_CIPCode')):
            return 2
    except:
        return 1

#
# The number of valence electrons the atom would have if it were unbonded and
# neutral
# TODO: Move this dictionary somewhere else. 
def GetNumValenceElectrons(atom):
    valence = {1: ['H', 'Li', 'Na', 'K', 'Rb', 'Cs', 'Fr'], # Alkali Metals
               2: ['Be', 'Mg', 'Ca', 'Sr', 'Ba', 'Ra'], # Alkali Earth Metals
               #transition metals???
               3: ['B', 'Al', 'Ga', 'In', 'Tl', 'Nh'], #
               4: ['C', 'Si', 'Ge', 'Sn', 'Pb', 'Fl'],
               5: ['N', 'P', 'As', 'Sb', 'Bi', 'Mc'], # Pnictogens
               6: ['O', 'S', 'Se', 'Te', 'Po', 'Lv'], # Chalcogens
               7: ['F', 'Cl', 'Br', 'I', 'At', 'Ts'], # Halogens
               8: ['He', 'Ne', 'Ar', 'Kr', 'Xe', 'Rn', 'Og']} # Noble Gases
    for k in valence:
        if atom.GetSymbol() in valence[k]:
            return k
    return 0

#
# Represents the total number of bonds to other atoms with V_i*b_i > 1, so
# basically bonds to atoms other than Hydrogen
#
# Here we can leverage the fact that RDKit does not even report Hydrogens by
# default to simply loop over the bonds. We will have to account for molecules
# that have hydrogens turned on before we can submit this code as a patch
# though.
#
# TODO: Create a dictionary for atom-B value pairs for use when AROMATIC is detected in bonds.
def GetBottcherBondIndex(atom):
    b_sub_i_ranking = 0
    bonds = []
    for bond in atom.GetBonds():
        bonds.append(str(bond.GetBondType()))
    for bond in bonds:
        if bond == 'SINGLE':
            b_sub_i_ranking += 1
        if bond == 'DOUBLE':
            b_sub_i_ranking += 2
        if bond == 'TRIPLE':
            b_sub_i_ranking += 3
    if 'AROMATIC' in bonds:
        # This list can be expanded as errors arise.
        if atom.GetSymbol() == 'C':
            b_sub_i_ranking += 3
        elif atom.GetSymbol() == 'N':
            b_sub_i_ranking += 2
    return b_sub_i_ranking

def GetBottcherComplexity(smiles, debug=False):
    themol=Chem.MolFromSmiles(smiles)
    complexity = 0
    Chem.AssignStereochemistry(themol,cleanIt=True,force=True,flagPossibleStereoCenters=True)
    atoms = themol.GetAtoms();
    atom_stereo_classes = []
    atoms_corrected_for_symmetry = []
    for atom in atoms:
        if atom.GetProp('_CIPRank') in atom_stereo_classes:
            continue
        else:
            atoms_corrected_for_symmetry.append(atom)
            atom_stereo_classes.append(atom.GetProp('_CIPRank'))
    for atom in atoms_corrected_for_symmetry:
        d = GetChemicalNonequivs(atom, themol)
        e = GetBottcherLocalDiversity(atom)
        s = GetNumIsomericPossibilities(atom)
        V = GetNumValenceElectrons(atom)
        b = GetBottcherBondIndex(atom)
        complexity += d*e*s*math.log(V*b,2)
        if debug:
            print(str(atom.GetSymbol()))
            print('\tSymmetry Class: ' + str(atom.GetProp('_CIPRank')))
            print('\tNeighbors: ')
            print('\tBonds: ')
            print('\tCurrent Parameter Values:')
            print('\t\td_sub_i: ' + str(d))
            print('\t\te_sub_i: ' + str(e))
            print('\t\ts_sub_i: ' + str(s))
            print('\t\tV_sub_i: ' + str(V))
            print('\t\tb_sub_i: ' + str(b))
    if debug:
        print('Current Complexity Score: ' + str(complexity))
        return
    return complexity

In [6]:
GetBottcherComplexity("COC1=C(O)C=C(CC(=O)O)C=C1Br")

161.80418485421137

#### 4.Proudfoot:

In [7]:
def calculate_path_complexity(smiles):
    # Convert SMILES to RDKit molecule object
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        print(f"Invalid SMILES: {smiles}")
        return None

    # Generate atom paths and path frequencies using Morgan fingerprint
    radius = 2  # Path length (corresponding to ECFP4)
    fingerprint = rdMolDescriptors.GetMorganFingerprint(mol, radius)
    paths = fingerprint.GetNonzeroElements()

    # Calculate path frequencies per atom environment
    atom_paths = defaultdict(list)
    for path, count in paths.items():
        atoms_in_path = path % mol.GetNumAtoms()  # Map path back to atom
        atom_paths[atoms_in_path].append(count)

    # Step 1: Calculate atomic complexity (CA)
    ca_values = {}
    for atom, path_counts in atom_paths.items():
        total_paths = sum(path_counts)
        path_fractions = [count / total_paths for count in path_counts]
        ca = -sum(p * math.log2(p) for p in path_fractions) + math.log2(total_paths)
        ca_values[atom] = ca

    # Step 2: Calculate molecular complexity (CM)
    cm = sum(ca_values.values())

    # Step 3: Calculate log-sum complexity (CM*)
    # cm_star = math.log2(sum(2 ** ca for ca in ca_values.values()))

    # Step 4: Calculate structural entropy complexity (CSE)
    # atom_types = [atom.GetAtomicNum() for atom in mol.GetAtoms()]
    # total_atoms = len(atom_types)
    # type_frequencies = {atype: atom_types.count(atype) / total_atoms for atype in set(atom_types)}
    # cse = -sum(freq * math.log2(freq) for freq in type_frequencies.values())

    return cm
        #"CA (per atom)": ca_values,
        #"CM (molecular complexity)": cm
        #"CM* (log-sum complexity)": cm_star,
        #"CSE (structural entropy complexity)": cse


In [8]:
calculate_path_complexity("COC1=C(O)C=C(CC(=O)O)C=C1Br")

30.54277674961796

#### 5.SPS:

In [9]:
# TODO: SPS calculation
def sps(smiles):
    mol = Chem.MolFromSmiles(smiles)
    sps = SPS(mol, normalize=False)
    return sps

In [10]:
sps("COC1=C(O)C=C(CC(=O)O)C=C1Br")

138

#### 6.nSPS:

In [11]:
# TODO: nSPS calculation
def nsps(smiles):
    mol = Chem.MolFromSmiles(smiles)
    nsps = SPS(mol, normalize=True)
    return nsps

In [12]:
nsps("COC1=C(O)C=C(CC(=O)O)C=C1Br")

9.857142857142858

#### 7.SAscore:

In [13]:
# TODO: Calculate the SAS
def sascore(smiles):
    mol=Chem.MolFromSmiles(smiles)
    sas_score = sascorer.calculateScore(mol)
    return sas_score

In [14]:
sascore("COC1=C(O)C=C(CC(=O)O)C=C1Br")

2.180114265525818

#### 8.SCS:

In [15]:
'''
This is a standalone, importable SCScorer model. It does not have tensorflow as a
dependency and is a more attractive option for deployment. The calculations are
fast enough that there is no real reason to use GPUs (via tf) instead of CPUs (via np)
'''

project_root = '/SCS/scscore-master'  # Update this to your project root directory

score_scale = 5.0
min_separation = 0.25

FP_len = 1024
FP_rad = 2

def sigmoid(x):
  return 1 / (1 + math.exp(-x))

class SCScorer():
    def __init__(self, score_scale=score_scale):
        self.vars = []
        self.score_scale = score_scale
        self._restored = False

    def restore(self, weight_path=os.path.join(project_root, 'models', 'full_reaxys_model_1024bool', 'model.ckpt-10654.as_numpy.pickle'), FP_rad=FP_rad, FP_len=FP_len):
        self.FP_len = FP_len; self.FP_rad = FP_rad
        self._load_vars(weight_path)
        #print('Restored variables from {}'.format(weight_path))

        if 'uint8' in weight_path or 'counts' in weight_path:
            def mol_to_fp(self, mol):
                if mol is None:
                    return np.array((self.FP_len,), dtype=np.uint8)
                fp = AllChem.GetMorganFingerprint(mol, self.FP_rad, useChirality=True) # uitnsparsevect
                fp_folded = np.zeros((self.FP_len,), dtype=np.uint8)
                for k, v in six.iteritems(fp.GetNonzeroElements()):
                    fp_folded[k % self.FP_len] += v
                return np.array(fp_folded)
        else:
            def mol_to_fp(self, mol):
                if mol is None:
                    return np.zeros((self.FP_len,), dtype=np.float32)
                return np.array(AllChem.GetMorganFingerprintAsBitVect(mol, self.FP_rad, nBits=self.FP_len,
                    useChirality=True), dtype=np.bool_)
        self.mol_to_fp = mol_to_fp

        self._restored = True
        return self

    def smi_to_fp(self, smi):
        if not smi:
            return np.zeros((self.FP_len,), dtype=np.float32)
        return self.mol_to_fp(self, Chem.MolFromSmiles(smi))

    def apply(self, x):
        if not self._restored:
            raise ValueError('Must restore model weights!')
        # Each pair of vars is a weight and bias term
        for i in range(0, len(self.vars), 2):
            last_layer = (i == len(self.vars)-2)
            W = self.vars[i]
            b = self.vars[i+1]
            x = np.matmul(x, W) + b
            if not last_layer:
                x = x * (x > 0) # ReLU
        x = 1 + (score_scale - 1) / (1 + math.exp(-x))
        return x

    def get_score_from_smi(self, smi='', v=False):
        if not smi:
            return ('', 0.)
        fp = np.array((self.smi_to_fp(smi)), dtype=np.float32)
        if sum(fp) == 0:
            if v: print('Could not get fingerprint?')
            cur_score = 0.
        else:
            # Run
            cur_score = self.apply(fp)
            if v: print('Score: {}'.format(cur_score))
        mol = Chem.MolFromSmiles(smi)
        if mol:
            smi = Chem.MolToSmiles(mol, isomericSmiles=True, kekuleSmiles=True)
        else:
            smi = ''
        return (smi, cur_score)

    def _load_vars(self, weight_path):
        if weight_path.endswith('pickle'):
            import cPickle as pickle
            with open(weight_path, 'rb') as fid:
                self.vars = pickle.load(fid)
                self.vars = [x.tolist() for x in self.vars]
        elif weight_path.endswith('json.gz'):
            with gzip.GzipFile(weight_path, 'r') as fin:    
                json_bytes = fin.read()                      
                json_str = json_bytes.decode('utf-8')            
                self.vars = json.loads(json_str)
                self.vars = [np.array(x) for x in self.vars]


In [16]:
# Define your SMILES string
smi = "COC1=C(O)C=C(CC(=O)O)C=C1Br"

# Create a DataFrame
df = pd.DataFrame({'SMILES': [smi]})

df

,SMILES
0,COC1=C(O)C=C(CC(=O)O)C=C1Br


In [17]:
if __name__ == '__main__':

    # Ensure the input file has at least one column
    if 'SMILES' not in df.columns:
        raise ValueError("Input file must have a 'smiles' column")

    # Initialize the model
    model = SCScorer()
    model.restore(os.path.join(project_root, 'models', 'full_reaxys_model_1024bool', 'model.ckpt-10654.as_numpy.json.gz'))

    # Process each SMILES string and store the scores
    scores = []
    for smi in df['SMILES']:
        _, sco = model.get_score_from_smi(smi)
        scores.append(sco)

    # Add a new column to the DataFrame for the scores
    df['SCS'] = scores
df

/tmp/ipykernel_1153771/451110517.py:65: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  x = 1 + (score_scale - 1) / (1 + math.exp(-x))


,SMILES,SCS
0,COC1=C(O)C=C(CC(=O)O)C=C1Br,2.334439
